In [1]:
!pip install mediapipe opencv-python

In [2]:
import os 
#current dir
cwd = os.getcwd()
print(cwd)

C:\Users\Aaron\Desktop\MasterProjects


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

![title](pose_tracking_full_body_landmarks.png)

In [4]:
#Calculate Angle function learned from Nicholas Renotte
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #Convert to the redian and then convert to degree
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    #If the angle is larger than 360, convert back to degree below 180
    if angle >180.0:
        angle = 360-angle
        
    return angle

def calculate_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    distance = abs(np.sqrt((a[0]-b[0])*(a[0]-b[0]) + (a[1]-b[1])*(a[1]-b[1])))
    return distance

def find_angles(landmarks, threshold):
    angle_array = []
    L_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    L_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    L_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    L_hip =   [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,  landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    L_knee =  [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    L_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    
    R_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    R_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    R_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    R_hip =   [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    R_knee =  [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    R_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    
    
    #LS will be left shoulder, and LE will be left elbow...
    if threshold == 0:
        #In this case, we only care about the upper body
        #angle array will have the order of:
        # 0, Left Shoulder(LS), Elbow(LE), Wrist(LW)
        # 1, LE, LS, LH
        # 2  RS, RE, RW
        # 3  RE, RS, RH
        angle_array.append(calculate_angle(L_shoulder, L_elbow, L_wrist))
        angle_array.append(calculate_angle(L_elbow, L_shoulder, L_hip))
        angle_array.append(calculate_angle(R_shoulder, R_elbow, R_wrist))
        angle_array.append(calculate_angle(R_elbow, R_shoulder, R_hip))
    if threshold == 1:
        # In this case, we care about the whole body
        # angle array will have the order of:
        # 0, Left Shoulder(LS), Elbow(LE), Wrist(LW)
        # 1, LE, LS, LH
        # 2  RS, RE, RW
        # 3  RE, RS, RH
        # 4  LS, LH, LK
        # 5  LH, LK, LA
        # 6  RS, RH, RK
        # 7  RH, RK, RA
        angle_array.append(calculate_angle(L_shoulder, L_elbow, L_wrist))
        angle_array.append(calculate_angle(L_elbow, L_shoulder, L_hip))
        angle_array.append(calculate_angle(R_shoulder, R_elbow, R_wrist))
        angle_array.append(calculate_angle(R_elbow, R_shoulder, R_hip))
        
        angle_array.append(calculate_angle(L_shoulder, L_hip, L_knee))
        angle_array.append(calculate_angle(L_hip, L_knee, L_ankle))
        angle_array.append(calculate_angle(R_shoulder, R_hip, R_knee))
        angle_array.append(calculate_angle(R_hip, R_knee, R_ankle))
    return angle_array

def find_distance(landmarks):
    L_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    R_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    distance = calculate_distance(L_wrist, R_wrist)
    return distance

def similarity_scoring(angle_array_sample, angle_array_in, hand_distance, penalty):
    deduction = 0.0
    #decuct from 100
    #
    for i in range(len(angle_array_sample)):
        deduction +=abs(angle_array_sample[i] - angle_array_in[i])*penalty
        #print(deduction)
    deduction += distance*10
    deduction *= penalty
    score = 100 - deduction
    if score < 0:
        score = 0
    return score



In [15]:
#Capture the video and the camera
cap_video = cv2.VideoCapture("test4_1.mp4")
cap_camera = cv2.VideoCapture("test4_1.mp4")
threshold = 1
counter = 0
score = 0
with mp_pose.Pose(model_complexity = 0, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap_video.isOpened() or cap_camera.isOpened():
        ret1, frame1 = cap_video.read()
        ret2, frame2 = cap_camera.read()
        if ret1:
             # Recolor image to RGB
            image1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
            image1 = cv2.flip(image1,1)
            image1.flags.writeable = False

            # Make detection
            results1= pose.process(image1)

            # Recolor back to BGR
            image1.flags.writeable = True
            image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
            try:               
                landmarks = results1.pose_landmarks.landmark
                # Save the sample frame parameters
                sample = find_angles(landmarks, threshold)
                distance_sample = find_distance(landmarks)
                cv2.putText(image1, str(distance_sample), 
                                   (50, 90), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                
            except:
                pass
            # Draw the landmarks
            mp_drawing.draw_landmarks(image1, results1.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                   mp_drawing.DrawingSpec(color=(226,105,65), thickness=2, circle_radius=2), 
                                   mp_drawing.DrawingSpec(color=(65,66,226), thickness=2, circle_radius=2) 
                                    )  
            cv2.imshow("video", image1)
        if ret2:
            image2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
            image2 = cv2.flip(image2,1)
            image2.flags.writeable = False

            # Make detection
            results2= pose.process(image2)

            # Recolor back to BGR
            image2.flags.writeable = True
            image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2BGR)

            try:
                landmarks = results2.pose_landmarks.landmark
                # Get your pose parameters
                angle = find_angles(landmarks, threshold)
                distance_cam = find_distance(landmarks)
                # Find Differences and get the score
                distance = abs(distance_cam - distance_sample)
                cur_score = similarity_scoring(sample, angle, distance, 0.75)
                score += cur_score
                counter += 1
                print(score/counter)
                cv2.putText(image2, str(cur_score), 
                                   (50,90), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                cv2.putText(image2, str(int(score/counter)), 
                                   (50,70), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA
                                    )
            except:
                pass
 
            cv2.imshow("camera", image2)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


cap_video.release()
cap_camera.release()
cv2.destroyAllWindows()

97.64576676454773
98.38867107373578
98.48987945271297
98.44665783709007
98.25587444980718
98.35923655433203
98.28589908715377
98.10681714345556
98.17077651525724
98.13104934878272
98.13669032120924
97.89859075795293
97.91403351178808
97.92914153947451
97.88512870053414
97.75050253260991
97.70354362822674
97.52286930471875
97.44589528267409
97.38365272514935
97.36894859199845
97.4256823826761
97.507871676262
97.4953627273278
97.43452458144125
97.42462545057919
97.45310695780375
97.48740082817919
97.50663741079747
97.55953590677726
97.59034037350949
97.61927719160333
97.5934379939865
97.62235341082159
97.6584717293015
97.6227113249492
97.56027104308444
97.5865498051531
97.60491217527593
97.61901420906375
97.59406506258335
97.59814930895726
97.61356671375079
97.63421914411119
97.63716839506665
97.60408139556517
97.56850060342228
97.55245317598973
97.527817184733
97.5236872542203
97.54353059177299
97.56089405463163
97.57915998242581
97.60516158262763
97.63286884386112
97.65167133307652
97.

In [11]:
angle = find_angles(landmarks, 0)
print(angle)

[44.97128522949177, 85.32486679330607, 12.649006913688316, 32.49795395153091]
